<center>
<img src='./img/nsidc_logo.png'/>

# **Using Coiled and h5coro to Produce ICESat-2 Sea Ice Height Time Series**

</center>

---

## **1. Tutorial Introduction/Overview**

Tutorial designed for the "DAAC data access in the cloud hands-on experience" session at the 2023 NSIDC DAAC User Working Group (UWG) Meeting. This is a copy of the `2_ATL07_timeseries` notebook for use with Coiled.


TODOS:
* Explain Coiled
* Question for Luis: Why would I use the decorator function (` @coiled.function()`) vs:

```
cluster = coiled.Cluster(n_workers=20, region="us-west-2")
client = cluster.get_client()
client
```
* How do we incorporate https://medium.com/coiled-hq/processing-a-250-tb-dataset-with-coiled-dask-and-xarray-574370ba5bde ? 


### Installing last versions from earthaccess and coiled

**NOTE**: Restart the kernel and clean output after the next cell

In [1]:
%%capture 

!pip uninstall -y earthaccess
!pip install git+https://github.com/nsidc/earthaccess.git@main

## **2. Tutorial steps**

Resoruces: each granule is approx 60-120 MB, A month of data for the Ross ocean returns 59 granules ~4.6 GB. We should use an instance preferable double the memory of the aprox data size we use.

### **Import Packages**

In [2]:
# To force use of shapely
import os
os.environ['USE_PYGEOS'] = '0'

# For searching NASA data
import earthaccess

# For reading data, analysis and plotting
# Could be moved to reader script
import numpy as np
from h5coro import h5coro, s3driver
import pandas as pd
import geopandas as gpd
from pqdm.threads import pqdm

# For resampling
from affine import Affine

# Foir plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import hvplot.xarray



0.6.1


### Authenticate

We need to authenticate and get AWS token

In [ ]:
auth = earthaccess.login()

### **Search for ICESat-2 ATL07 data**

We use `earthaccess` to search for granules in the region of interest for the time period of interest.  

The region is set by name below.  Currently, we have two options: the Ross Sea, and the Southern Ocean and adjoining seas.

The range of dates is set by assigning a start year and end year to `year_begin` and `year_end`.  Setting `year_begin` and `year_end` to the same year retreives data for one year.

In [ ]:
# To avoid copying and pasting region tuples
region = "Ross Sea"  # Set region to "Ross Sea" for just Ross Sea or "Antarctica" for southern ocean 
ross_sea = (-180, -78, -160, -74)
antarctic = (-180, -90, 180, -60)
this_region = antarctic if region == "Antarctica" else ross_sea

year_begin = 2019
year_end = 2019

In [ ]:
atl10 = {}
total_results = 0

for year in range(year_begin, year_end+1):
    
    print(f"Searching year {year} ...")
    granules = earthaccess.search_data(
        short_name = 'ATL10',
        version = '006',
        cloud_hosted = True,
        bounding_box = this_region,
        temporal = (f'{year}-09-01',f'{year}-09-30'),
    )
    total_results += len(granules)
    atl10[str(year)] = granules
print(f"Total: {total_results}")

### Access the Granules

Because the CryoCloud is hub is running on servers in AWS region `us-west-2`, which is the same region as the NASA Earthdata Cloud, granules can be accessed directly without having to download the files first.  This is analogous to how you would work with files on your local filesystem.  However, _under the hood_ there are differences.

Initially, we load data for each year into a `geopandas.DataFrame`.  `geopandas` is an extension of the `pandas` package.  `pandas` is designed to work with `tabular` data - _think data you might put into a spreadsheet_.  `geopandas`, extends `pandas` to work with geospatial data by adding geometries (points, lines and polygons) and a coordinate reference system (CRS), so that data in each row is associated with a geospatial feature located on Earth.  ICESat-2 track data is well suited to the DataFrame data model because data are related to points or segments.  Once data is in a `geopandas.DataFrame`, the data can be reprojected and queried using methods you may be used to using in a GIS.

#### Open the files

The first step is to open the files using the `earthaccess.open` method.   So that we can work with files by year, we create a dictionary organized by year that contains _file-like_ objects.  We haven't read what is in the files, we've just opened them ready for reading.

In [ ]:
file_tree = {}
for year, granules in atl10.items():
    file_tree[year] = earthaccess.open(granules)

#### Read data into `geopandas.DataFrame`

The next step is to read the data and put it into a Dataframe.  We use `h5coro`, which is a package developed by the SlideRule project to efficiently read HDF5 files in the cloud.  Recall from the Cloud Optimized Format presentation, the HDF5 format and the HDF5 library for reading and writing those files are not well suited to accessing data in the cloud.  `h5coro` was developed to solve some of the problems related to HDF5 format and tools.  Using `h5coro` with `dask`, a python package for parallel processing on multicore local machines and distributed cluster in the cloud, is 5x faster at reading the ATL10 data than using the `h5py` package, an HDF5 reader that uses the HDF5 library.

The `read_atl10` is long, so we have put it in a module, which is imported into this notebook.  If your are interested, take a look at `readers.py`.  The main features of the function are briefly described here.

We follow the processing steps for ATL20 to generate our freeboard grids.  For each grid cell that contain one or more freeboard segments, a grid cell mean freeboard is calculated as a mean of `gtx/freeboard_segment/beam_fb_height` from ATL10, weighted by segment length `gtx/freeboard_segment/heights/height_segment_length_seg`.  To resample segments to grid cells, we also need the geodetic coordinates for each segment in `gtx/freeboard_segment/latitude` and `gtx/freeboard_segment/longitude`. As an additional locator, we also read `gtx/freeboard_segment/delta_time`.

The ATL20 gridded freeboards are calculated using only the _strong beams_ of each beam pair.  Depending on the orientation of the ICESat-2 satellite, left beams or right beams are the _strong beams_.  Satellite orientation is given in the `orbit_info/sc_orient` dataset.  The orientation could be forward or backward, or transition.  We only use data in forward or backward orientations.  In addition to orientation, we also need to know the number of GPS seconds between the GPS epoch
(1980-01-06T00:00:00.000000Z UTC) and the ATLAS Standard Data Product (SDP) epoch (2018-01-01:T00.00.00.000000 UTC).  This number of seconds must be added to `delta_time`.  This is stored in `ancillary_data/atlas_sdp_gps_epoch` 

```{note}
There are three beam pairs numbered 1, 2 and 3.  Each of these beam pairs has a left and right beam.  Beams are numbered `gt1l` and `gt1r`, `gt2l` and `gt2r`, and `gt3l` and `gt3r`.
```

`read_atl10` reads these Datasets.  Datasets containing segment data are stored in the `DATASETS` constant, which is a python `list`, in `reader.py`.  If you want additional datasets, these can be added to the list. 

In [ ]:
## Based on the READ function form Younghyun Koo for the sea ice tutorial at the IS2 hackweek

#from h5coro import h5coro, s3driver, filedriver
from itertools import product
#import geopandas as gpd
# import pandas as pd
# import numpy as np
# import gc
    
GPS_EPOCH = pd.to_datetime('1980-01-06 00:00:00')

def get_strong_beams(f):
    """Returns ground track for strong beams based on IS2 orientation"""
    orient  = f['orbit_info/sc_orient'][0]

    if orient == 0:
        return [f"gt{i}l" for i in [1, 2, 3]]
    elif orient == 1:
        return [f"gt{i}r" for i in [1, 2, 3]]
    else:
        raise KeyError("Spacecraft orientation neither forward nor backward")


def get_credentials(file):
    """Returns credentials dict with keys expected by h5coro
    
    TODO: could add as option for earthaccess
    """
    return {
        "aws_access_key_id": file.s3.storage_options["key"],
        "aws_secret_access_key": file.s3.storage_options["secret"],
        "aws_session_token": file.s3.storage_options["token"]
    }
    
    
def read_atl10_local(files, executors):
    """Returns a consolidated GeoPandas dataframe for a set of ATL10 file pointers.
    
    Parameters:
        files (list[S3FSFile]): list of authenticated fsspec file references to ATL10 on S3 (via earthaccess)
        executors (int): number of threads
    
    """
    def read_atl10(file):
        """Reads datasets required for creating gridded freeboard from a single ATL10 file
        
        file: an authenticated fsspec file reference on S3 (returned by earthaccess)
        
        returns: a list of geopandas dataframes
        """
        
        # Open file object
        f = h5coro.H5Coro(file.info()["name"], s3driver.S3Driver, credentials=get_credentials(file))
        
        # Get strong beams based on orientation
        ancillary_datasets = ["orbit_info/sc_orient", "ancillary_data/atlas_sdp_gps_epoch"]
        f.readDatasets(datasets=ancillary_datasets, block=True)
        strong_beams = get_strong_beams(f)
        atlas_sdp_gps_epoch = f["ancillary_data/atlas_sdp_gps_epoch"][:]
        
        # Create list of datasets to load
        datasets = ["freeboard_segment/latitude",
                    "freeboard_segment/longitude",
                    "freeboard_segment/delta_time",
                    "freeboard_segment/seg_dist_x",
                    "freeboard_segment/heights/height_segment_length_seg",
                    "freeboard_segment/beam_fb_height",
                    "freeboard_segment/heights/height_segment_type"]
        ds_list = ["/".join(p) for p in list(product(strong_beams, datasets))]
        # Load datasets
        f.readDatasets(datasets=ds_list, block=True)
        
        # Create a list of geopandas.DataFrames containing beams
        tracks = []
        for beam in strong_beams:
            ds = {dataset.split("/")[-1]: f[dataset][:] for dataset in ds_list if dataset.startswith(beam)}
            
            # Convert delta_time to datetime
            ds["delta_time"] = GPS_EPOCH + pd.to_timedelta(ds["delta_time"]+atlas_sdp_gps_epoch, unit='s')

            # Add beam identifier
            ds["beam"] = beam
            
            # Set fill values to NaN - assume 100 m as threshold
            ds["beam_fb_height"] = np.where(ds["beam_fb_height"] > 100, np.nan, ds["beam_fb_height"])
            
            geometry = gpd.points_from_xy(ds["longitude"], ds["latitude"])
            del ds["longitude"]
            del ds["latitude"]
            
            gdf = gpd.GeoDataFrame(ds, geometry=geometry, crs="EPSG:4326")
            gdf.dropna(axis=0, inplace=True)
            tracks.append(gdf)

#             gc.collect()
        return tracks
    
    df = pqdm(files, read_atl10, n_jobs=executors)
    combined = pd.concat([t[0] for t in df if type(t) is list])
    
    return combined

The idea would be to split each year into its own Dask worker

In [ ]:
%%time
tracks = read_atl10_local(file_tree["2019"], executors=16)

In [ ]:
tracks

In [ ]:
tracks.info(memory_usage='deep')  # what does this do?

### For future IO eficient operations we save the geodataframe as parquet

In [ ]:
tracks["delta_time"] = tracks["delta_time"].astype('datetime64[s]')
tracks.info()

In [ ]:
#tracks.to_parquet("atl10-2019.parquet")

#### Geopandas Read function 

The function below extracts latitude, longitude, segment distance, segment length, surface type, and freeboard height. See the [NSIDC's ATL10 User Guide](https://nsidc.org/sites/default/files/documents/user-guide/atl10-v006-userguide.pdf) for more details on these variables.

## Grid track data

This follows the processing steps described in the ATL20 - Gridded Sea Ice Freeboard - ATBD but gridding to a EASE-Grid v2 6.25 km grid.  Any projected coordinate system or grid could be chosen.  The procedure could be modified with extra QC steps or modifications.  **The world is your oyster - or [Aplacophoran](https://antarcticsun.usap.gov/science/4447/)**.

The processing steps are:

- remove non-ice and low quality segments 
- resample freeboard segments to a grid
- calculate aggregate statistics
    + mean segment length
    + segment count
    + length weighted mean freeboard
    + length weighted standard deviation of freeboard
    
#### Grid Cell Mean Segment Length $\bar{L}$

$$
\bar{L}(x, y, D) = \frac{\sum L_i}{N}
$$

where $L_i$ is `/gtx/freeboard_beam_segment/height_segments/height_segment_length_seg`, $x$ and $y$ are projected coordinates for grid centers, and $D$ is day. 

#### Grid Cell Mean Freeboard $\bar{h}$

$$
\bar{h}(x, y, D) = \frac{\sum L_i h_i}{\sum L_i}
$$

where $h_i$ is `gtx/freeboard_beam_segment/beam_freeboard/beam_fb_height`.

#### Grid Cell Standard Deviation of Freeboard $\sigma^2 (x, y, D)$

$$
\sigma^2 (x, y, D) = \frac{\sum L_i (h_i)^2}{\sum L_i} - \bar{h}^2 (x, y, D)
$$

### Resample Freeboard Segments to a Grid

Following the ATL20 ATBD, we will use a _drop-in-the-bucket_ resampling scheme.  This is simple and relatively easy to implement.  More complex resampling schemes could be substituted.

To demonstrate resampling we will resample freeboard segments to WGS84 / NSIDC EASE-Grid v2.0 South with a grid resolution of 6.25 km.  The EPSG code for the WGS84 / NSIDC EASE-Grid South coordinate reference system is [6932](https://epsg.org/crs_6932/WGS-84-NSIDC-EASE-Grid-2-0-South.html).

We will use the standard 6.25 km grid.  To define the grid, we need the grid dimensions (nrows and ncols), the x and y projected coordinates of the upper-left corner of the upper-left grid cell, and the height and width of the grid cells in the same units as the projected coordinates.  In this case, the units are meters.

In [ ]:
easegrid2_epsg = 6932

nrow = 2880
ncol = 2880
upper_left_x = -9000000.0
upper_left_y = 9000000.0
width = 6250.0
height = -6250.0

The first step is to reproject the points from geodetic coordinates (latitude and longitude) to projected coordinates (x, y).  Because the data are in a `geopandas.DataFrame` we can use the `to_crs` method.  This takes an EPSG code either as a numeric value (`6932`) or as a string (`"EPSG:6932"`).

You can see that the `POINT` objects in the `geometry` have changed from having latitudes and longitudes as coordinates to x and y in meters.

In [ ]:
%%time
tracks = tracks.to_crs(easegrid2_epsg)
tracks.head()

A _Drop-in-the-Bucket_ resampling scheme collects points into the grid cells that they intersect with, and then calculates aggregate statistics for each grid cell using attributes associated with those points.

We'll find the grid cell that contains each segment by calculating the row and column coordinates for each segment from the projected coordinates.  This is done by creating an _Affine_ transformation matrix for the grid.  The Affine matrix is just a matrix representation of the algebraic expressions to convert row and column indices of the grid to projected coordinates.  The equations below give the forward transformation from `(row, col)` to `(x, y)`. 

$$
x = width * col + upper\_left\_x \\
y = height * row + upper\_left\_y
$$

These are expressed in matrix form:

$$
\begin{bmatrix}
x \\
y \\
0
\end{bmatrix} = 
\begin{bmatrix}
a & 0 & c \\
0 & d & e \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
col \\
row \\
1
\end{bmatrix}
$$

where $a$ is $\mathsf{width}$, $c$ is $\mathsf{upper\_left\_x}$, $d$ is $height$, and $e$ is $upper\_left\_y$.

```{note}
The projected coordinate system we are using is a cartesian plane with the origin at the South Pole.  The `x` coordinates increase to the right, and `y` coordinates increase up.  For raster data, which includes grids and images, have the origin at the upper-left corner of the grid.  Column indices increase from right to left, and row indices increase from top to bottom.
```

We use the `affine` package to create a forward transformation matrix (`fwd`) using the grid parameters above.  To transform `(x, y)` projected coordinates to `(row, col)`, we can calculate the reverse transformation matrix using `~fwd`.

`(row, col)` coordinates are still rational numbers.  We want an integer row and column indices for grid cells.  We can use the `floor` function to get integers.  `row` and `column` indices are zero based.

We want to be able to leverage the `geopandas.Dataframe.groupby` functionality to collect points into grid cells, so we need a unique identifier to group the data.  We can calculate a unique cell index from `row` and `column` indices as follows:

$$
cell\_index = row * ncol + col
$$

This is encapsulated in the function `get_grid_index`.  This function is then applied to the `geometry` of tracks.

In [ ]:
def get_grid_index(xy):
    geotransform = (upper_left_x, width, 0., upper_left_y, 0., height)
    fwd = Affine.from_gdal(*geotransform)
    row, col = ~fwd * xy
    return (np.floor(row) * ncol) + np.floor(col)


... and applied to the coordinates of the `POINTS` 

In [ ]:
%%time
tracks["grid_index"] = [get_grid_index((x, y)) for x, y in zip(tracks.geometry.x, tracks.geometry.y)]

tracks.head()

In [ ]:
cell_segment_counts = tracks.groupby("grid_index")["beam_fb_height"].mean()

In [ ]:
cell_segment_counts

In [ ]:
grid = np.zeros((nrow, ncol)).flatten()
grid[cell_segment_counts.index.values.astype(int)] = cell_segment_counts
grid = grid.reshape((nrow, ncol))
#grid = np.where(grid > 0, grid, np.nan)

In [ ]:
grid.shape

In [ ]:
grid.min(), grid.max()

In [ ]:
%%matplotlib.
plt.imshow(grid)

In [ ]:
# Get bounding box for Ross Sea
ross_sea_gdf = gpd.read_file("ross_sea.json")
ross_sea_gdf.to_crs(easegrid2_epsg).bounds.values

In [ ]:
from cartopy.crs import AzimuthalEquidistant

class EASEGrid2South(AzimuthalEquidistant):
    
    def __init__(self):
        super(EASEGrid2South, self).__init__(central_longitude=0.0, central_latitude=-90.0,
                 false_easting=0.0, false_northing=0.0,
                 globe=None)
        
        self._bounds = [-9000000.0, -9000000.0, 9000000.0, 9000000.0]
        self._x_limits = self._bounds[0], self._bounds[2]
        self._y_limits = self._bounds[1], self._bounds[3]
        
    @property
    def threshold(self):
        return 1e5

    @property
    def x_limits(self):
        return self._x_limits

    @property
    def y_limits(self):
        return self._y_limits


In [ ]:
#proj = EASEGrid2South()
proj = AzimuthalEquidistant(central_latitude=-90)


fig = plt.figure(figsize=(10,10))
ax = fig.add_axes(111, projection=proj)

#ax.set_extent(

### **Assign to grid and calculate grid cell mean**

## **3. Learning outcomes recap (optional)**

Provide a brief summary of the learning outcomes of the tutorial


## **4. Additional resources (optional)**

List some additional resources for users to consult, if applicable/desired.

________

### **When your tutorial is ready for review,  please read our [Contributor Guide](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/contributor_guide.md) for next steps.**